In [1]:
import pandas as pd
from evaluation.generated_dataset import GeneratedDataset, load_all_from_config

In [2]:
all_datasets = load_all_from_config()

FileNotFoundError: [Errno 2] No such file or directory: '/home/kna/WyckoffTransformer/cache/mp_20/analysis_datasets/WyckoffTransformer/DiffCSP++.pkl.gz'